### Summation Notation

Linear programs typically involve sums over many decision variables. We introduce sigma notation ($\sum$) as a shorthand way to represent arbitrarily large sums.

Consider the set $I = \{1, 2, 3\}$. The expression 
\begin{equation}
\sum_{i \in I} a_i x_i
\end{equation}
is shorthand for the sum 
\begin{equation}
a_1 x_1 + a_2 x_2 + a_3 x_3.
\end{equation}
The symbol $\in$ is read as "in". 

What we gain from adopting this notation is that now if we consider $I = \{1, 2, \ldots, n\}$ for an arbitrary $n$,  $\sum_{i \in I} a_i x_i = a_1 x_1 + a_2 x_2 + \cdots + a_n x_n$. That is, we can write arbitrarily-sized sums in one shot.

### Knapsack problem

Suppose you are given a set of items $I = \{1, \ldots, n\}$, each with a value $v_i$ and a weight $w_i$. We have a knapsack that has a weight capacity of $b$. What is the highest valued collection of items that can go into the knapsack? Assume that you can put fractions of items into the knapsack.

Let $x_i, i \in I$ be the fraction of item $i$ that goes in the knapsack. We can use sigma notation to write expressions for the value and weight of our collection of items. The total value is given by $\sum_{i \in I} v_i x_i$ and the total weight is given by $\sum_{i \in I} w_i x_i$. Then we can formulate our problem as a linear program:

\begin{eqnarray}
\max_x && \sum_{i \in I} v_i x_i \\
\mbox{s.t.} && \sum_{i \in I} w_i x_i \le b \\
&& 0 \le x_i \le 1,\;\;i \in I.
\end{eqnarray}

Consider the following data:

In [1]:
weights = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
values = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
I = range(len(values))
capacity = 750
I

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

First, we're going to need to instantiate the decision variables and save references to them in a data structure. We'll use a Python list for this purpose.

In [2]:
from gurobipy import *

In [3]:
m = Model()

In [4]:
item_selected = []
for i in I:
    item_selected.append(m.addVar(ub=1, obj=values[i], name='item_selected.' + str(i)))

In [5]:
m.update()

Now we need to write the knapsack capacity constraint, which means we'll need an expression for the total weight of our collection of items (in terms of the decision variables we just created). We use a LinExpr object for this purpose, and we've got a few options to build up this object. The first uses some syntactic sugar that is only available through the Python API, namely the quicksum method.

In [6]:
quicksum(weights[i]*item_selected[i] for i in I)

<gurobi.LinExpr: 70.0 item_selected.0 + 73.0 item_selected.1 + 77.0 item_selected.2 + 80.0 item_selected.3 + 82.0 item_selected.4 + 87.0 item_selected.5 + 90.0 item_selected.6 + 94.0 item_selected.7 + 98.0 item_selected.8 + 106.0 item_selected.9 + 110.0 item_selected.10 + 113.0 item_selected.11 + 115.0 item_selected.12 + 118.0 item_selected.13 + 120.0 item_selected.14>

A slightly less slick way to acheive this is through a for loop.

In [7]:
total_weight = LinExpr()
for i in I:
    total_weight += weights[i]*item_selected[i]
print total_weight

<gurobi.LinExpr: 70.0 item_selected.0 + 73.0 item_selected.1 + 77.0 item_selected.2 + 80.0 item_selected.3 + 82.0 item_selected.4 + 87.0 item_selected.5 + 90.0 item_selected.6 + 94.0 item_selected.7 + 98.0 item_selected.8 + 106.0 item_selected.9 + 110.0 item_selected.10 + 113.0 item_selected.11 + 115.0 item_selected.12 + 118.0 item_selected.13 + 120.0 item_selected.14>


Here, the += operator is overloaded to return a LinExpr object as opposed to doing arithmetic. Operator overloading isn't available in Java, and is not advisable in .NET. Those API's provide methods on the LinExpr (really GRBLinExpr) object that do the same job.

In [8]:
total_weight = LinExpr()
for i in I:
    total_weight.add(item_selected[i], weights[i])
print total_weight

<gurobi.LinExpr: 70.0 item_selected.0 + 73.0 item_selected.1 + 77.0 item_selected.2 + 80.0 item_selected.3 + 82.0 item_selected.4 + 87.0 item_selected.5 + 90.0 item_selected.6 + 94.0 item_selected.7 + 98.0 item_selected.8 + 106.0 item_selected.9 + 110.0 item_selected.10 + 113.0 item_selected.11 + 115.0 item_selected.12 + 118.0 item_selected.13 + 120.0 item_selected.14>


However we build up the summation of the weights, we can use the LinExpr object as the left-hand side of a constraint.

In [9]:
weight_con = m.addConstr(total_weight <= capacity, name='total_weight')

In [10]:
m.update()

In [ ]:
m.write('knapsack.lp')

In [ ]:
!more knapsack.lp

In [11]:
m.optimize()

Optimize a model with 1 rows, 15 columns and 15 nonzeros
Coefficient statistics:
  Matrix range    [7e+01, 1e+02]
  Objective range [1e+02, 2e+02]
  Bounds range    [1e+00, 1e+00]
  RHS range       [8e+02, 8e+02]
Presolve removed 1 rows and 15 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00


Remember that Gurobi will minimize by default...

In [12]:
m.ModelSense = GRB.MAXIMIZE

In [13]:
m.optimize()

Optimize a model with 1 rows, 15 columns and 15 nonzeros
Coefficient statistics:
  Matrix range    [7e+01, 1e+02]
  Objective range [1e+02, 2e+02]
  Bounds range    [1e+00, 1e+00]
  RHS range       [8e+02, 8e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7560000e+03   5.335938e+00   0.000000e+00      0s
       1    1.4615043e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.461504348e+03


In [14]:
for var in item_selected:
    print var.VarName, var.X

item_selected.0 1.0
item_selected.1 0.0
item_selected.2 1.0
item_selected.3 0.0
item_selected.4 0.0
item_selected.5 0.0
item_selected.6 1.0
item_selected.7 1.0
item_selected.8 1.0
item_selected.9 0.0
item_selected.10 0.0
item_selected.11 0.0
item_selected.12 0.721739130435
item_selected.13 1.0
item_selected.14 1.0


In [15]:
for var in item_selected:
    var.vtype = GRB.BINARY

In [16]:
m.optimize()

Optimize a model with 1 rows, 15 columns and 15 nonzeros
Coefficient statistics:
  Matrix range    [7e+01, 1e+02]
  Objective range [1e+02, 2e+02]
  Bounds range    [1e+00, 1e+00]
  RHS range       [8e+02, 8e+02]
Found heuristic solution: objective 1249
Presolve time: 0.00s
Presolved: 1 rows, 15 columns, 15 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)

Root relaxation: objective 1.461504e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1461.50435    0    1 1249.00000 1461.50435  17.0%     -    0s
H    0     0                    1455.0000000 1461.50435  0.45%     -    0s
H    0     0                    1458.0000000 1461.50435  0.24%     -    0s
     0     0 1460.43836    0    1 1458.00000 1460.43836  0.17%     -    0s
     0     0 1460.38710    0    3 1458.00000 1460.38710  0.16%     -    0s
     0     2 1460.38710    0    3

In [17]:
for var in item_selected:
    print var.VarName, var.X

item_selected.0 1.0
item_selected.1 0.0
item_selected.2 1.0
item_selected.3 -0.0
item_selected.4 1.0
item_selected.5 -0.0
item_selected.6 1.0
item_selected.7 1.0
item_selected.8 1.0
item_selected.9 -0.0
item_selected.10 -0.0
item_selected.11 -0.0
item_selected.12 -0.0
item_selected.13 1.0
item_selected.14 1.0


We can wrap this up in a method that takes in the data and returns a Model.

In [18]:
def get_knapsack_model(capacity, weights, values):
    items = range(len(weights))
    m = Model()
    m.ModelSense = GRB.MAXIMIZE
    item_selected = [m.addVar(ub=1, obj=values[item], name="item_selected." + str(item))
                              for item in items]
    m.update()
    m.addConstr(quicksum(weights[item]*item_selected[item] for item in items) <= capacity)
    m.update()
    return m

Let's try this with a different set of inputs:

In [19]:
m = get_knapsack_model(capacity=104,
                       weights=[25, 35, 45, 5, 25, 3, 2, 2],
                       values=[350, 400, 450, 20, 70, 8, 5, 5])

In [20]:
m.optimize()

Optimize a model with 1 rows, 8 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [2e+00, 4e+01]
  Objective range [5e+00, 4e+02]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+02, 1e+02]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3080000e+03   5.937500e-01   0.000000e+00      0s
       1    1.1900000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.190000000e+03


In [21]:
for var in m.getVars():
    print var.VarName, var.X

item_selected.0 1.0
item_selected.1 1.0
item_selected.2 0.977777777778
item_selected.3 0.0
item_selected.4 0.0
item_selected.5 0.0
item_selected.6 0.0
item_selected.7 0.0


We can see that we selected the first two items and 97.7% of the third. What if we aren't allowed to include fractions of items in the knapsack?

In [22]:
for var in m.getVars():
    var.vtype = GRB.BINARY

In [23]:
m.optimize()

Optimize a model with 1 rows, 8 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [2e+00, 4e+01]
  Objective range [5e+00, 4e+02]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+02, 1e+02]
Found heuristic solution: objective 858
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 1 rows, 7 columns, 7 nonzeros
Variable types: 0 continuous, 7 integer (6 binary)

Root relaxation: objective 1.188571e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1188.57143    0    1  858.00000 1188.57143  38.5%     -    0s
H    0     0                     888.0000000 1188.57143  33.8%     -    0s
H    0     0                     898.0000000 1188.57143  32.4%     -    0s
H    0     0                     900.0000000 1188.57143  32.1%     -    0s

Cutting planes:
  Cover: 1

Explored 0 nodes (1 simplex iterations) in 0.02 

In [24]:
for var in m.getVars():
    print var.VarName, var.X

item_selected.0 1.0
item_selected.1 -0.0
item_selected.2 1.0
item_selected.3 1.0
item_selected.4 1.0
item_selected.5 -0.0
item_selected.6 1.0
item_selected.7 1.0


Now we've really solved an integer program, and it's worth noting that the solution is considerably different than that of the linear programming version of the problem.